### [Reference](https://dl.acm.org/doi/10.1145/3289600.3291015)

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!nvidia-smi

Sun Apr 23 15:20:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# warning shit up
import warnings
warnings.filterwarnings("ignore")

In [9]:
%cd CAN/CAN-Pytorch

/content/CAN/CAN-Pytorch


In [10]:
train_df = pd.read_csv('/content/gdrive/MyDrive/hw2_data/dataset3/train.csv')
print(train_df.describe())
print(train_df.head())
print()
print(f"There are {train_df[train_df['label']==1]['id'].count()} edges in training graph.")
print(f"There are {train_df['to'].unique().max()} vertices in the training graph.")

                to         from        label
count  2572.000000  2572.000000  2572.000000
mean    439.468896   447.962286     0.494946
std     245.571595   251.356563     0.500072
min       1.000000     0.000000     0.000000
25%     235.000000   227.000000     0.000000
50%     445.000000   453.500000     0.000000
75%     657.250000   674.000000     1.000000
max     875.000000   875.000000     1.000000
      id   to  from  label
0  E2143  612    59      1
1  E1615  843   189      1
2  E2103  677   416      1
3  E3131  797   506      0
4  E1257  175   856      0

There are 1273 edges in training graph.
There are 875 vertices in the training graph.


In [11]:
with open("/content/gdrive/MyDrive/hw2_data/dataset3/content.csv", 'r') as f:
    lines = f.readlines()
    attributes = [ [] for _ in range(len(lines)) ]
    for line in lines:
        line = line.split()
        line = list( map(int, line) )
        attributes[line[0]].extend(line[1:])

attributes = np.array(attributes)
print(f"There are {attributes.shape[0]} nodes")
print(f"There are {attributes.shape[1]} attributes per node")

There are 877 nodes
There are 1703 attributes per node


In [12]:
node_num = attributes.shape[0]
edge_num = train_df[train_df['label']==1]['id'].count()
attr_num = attributes.shape[1]

In [14]:
# .edge format
# #Nodes node_num
# #Edges edge_num
# node1, node2
# ...

# .attr format
# #Nodes node_num
# #Edges edge_num
# node_num
# attribute_num
# node attr

def reconstruct_dataset(node_num, edge_num, attr_num, edges, attr):
    file_path = "data/test"
    edge_f = open(file_path + '.edge', 'w')
    attr_f = open(file_path + '.node', 'w')
    node_num, edge_num, attr_num = str(node_num), str(edge_num), str(attr_num)
    # construct edge file
    edge_f.write("#Nodes")
    edge_f.write("\t")
    edge_f.write(node_num)
    edge_f.write("\n")
    edge_f.write("#Edges")
    edge_f.write("\t")
    edge_f.write(edge_num)
    edge_f.write("\n")
    for idx in edges:
        n1, n2 = str(idx[0]), str(idx[1])
        edge_f.write(n1)
        edge_f.write("\t")
        edge_f.write(n2)
        edge_f.write("\n")
    edge_f.close()

    # attributes from one hot to index
    attributes = list(np.argwhere(attr== 1))
    # construct attr file
    attr_f.write("#Nodes")
    attr_f.write("\t")
    attr_f.write(node_num)
    attr_f.write("\n")
    attr_f.write("#Attributes")
    attr_f.write("\t")
    attr_f.write(attr_num)
    attr_f.write("\n")
    for idx in attributes:
        n, attr = str(idx[0]), str(idx[1])
        attr_f.write(n)
        attr_f.write("\t")
        attr_f.write(attr)
        attr_f.write("\n")
    attr_f.close()

reconstruct_dataset(node_num, edge_num, attr_num, train_df[train_df['label']==1][['to', 'from']].to_numpy(), attributes)

In [15]:
!python train.py --dataset 'test'

dataset:test, node_num:877,edge_num:1273,attribute_num:1703
/content/CAN/CAN-Pytorch/train.py:107: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  preds_all = np.hstack([preds, preds_neg]).astype(np.float)
Epoch: 0001 train_loss= 4.42720 log_lik= 4.42352 KL= -0.00368 train_acc= 0.97805 val_edge_roc= 0.53893 val_edge_ap= 0.56439 val_attr_roc= 0.49134 val_attr_ap= 0.50596 time= 3.50969
/content/CAN/CAN-Pytorch/train.py:107: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` 

In [16]:
!python final_train.py --dataset test

dataset:test, node_num:877,edge_num:1273,attribute_num:1703
Epoch: 0001 train_loss= 4.63428 log_lik= 4.62955 KL= -0.00472 train_acc= 0.96917 time= 0.60104
Epoch: 0002 train_loss= 6.54652 log_lik= 6.52689 KL= -0.01963 train_acc= 0.72144 time= 0.02246
Epoch: 0003 train_loss= 4.08497 log_lik= 4.06736 KL= -0.01762 train_acc= 0.74387 time= 0.02472
Epoch: 0004 train_loss= 3.91539 log_lik= 3.89838 KL= -0.01701 train_acc= 0.85893 time= 0.02152
Epoch: 0005 train_loss= 3.15615 log_lik= 3.13709 KL= -0.01906 train_acc= 0.82225 time= 0.02290
Epoch: 0006 train_loss= 2.82292 log_lik= 2.79965 KL= -0.02327 train_acc= 0.78571 time= 0.02228
Epoch: 0007 train_loss= 2.57741 log_lik= 2.54896 KL= -0.02845 train_acc= 0.82875 time= 0.02611
Epoch: 0008 train_loss= 2.41777 log_lik= 2.38452 KL= -0.03326 train_acc= 0.89935 time= 0.02515
Epoch: 0009 train_loss= 2.03293 log_lik= 1.99512 KL= -0.03781 train_acc= 0.97293 time= 0.02250
Epoch: 0010 train_loss= 1.92287 log_lik= 1.87923 KL= -0.04364 train_acc= 0.88192 time

In [17]:
#　for testing the final links
def reparameterize(z_u_mean,z_u_log_std,z_a_mean,z_a_log_std): 
    z_u = z_u_mean + torch.randn_like(z_u_log_std) * torch.exp(z_u_log_std)
    z_a = z_a_mean + torch.randn_like(z_a_log_std) * torch.exp(z_a_log_std)
    return z_u,z_a
def sigmoid(x):
    x = np.clip(x, -500, 500)
    return 1.0 / (1 + np.exp(-x))

In [18]:
# load test data
test_df = pd.read_csv("/content/gdrive/MyDrive/hw2_data/dataset3/test.csv")
# testing
print("Start testing...")

z_u_mean = torch.tensor(np.load('result/AGAE_test_n_mu.emb.npy'))
z_u_log_std = torch.tensor(np.load('result/AGAE_test_n_sig.emb.npy'))
z_a_mean = torch.tensor(np.load('result/AGAE_test_a_mu.emb.npy'))
z_a_log_std = torch.tensor(np.load('result/AGAE_test_a_sig.emb.npy'))
z_u, _ = reparameterize(z_u_mean,z_u_log_std,z_a_mean,z_a_log_std)
outputs = sigmoid( torch.mm(z_u, z_u.t()).detach().cpu().numpy() )

test_df['prob'] = pd.Series(outputs[test_df['to'].to_numpy(), test_df['from'].to_numpy()])
test_df = test_df.drop(columns=['to', 'from'])
# store the csv
test_df.to_csv('/content/gdrive/MyDrive/hw2_data/dataset3/upload.csv', index=False)
print("finish testing")

Start testing...
finish testing
